# Build a Q&A application with Bedrock, Llama Index and Langchain

This notebook explains steps requried to build a Question & Answer application with Bedrock - Anthropic Claude model, Langchain and LLama index

## Pre-requisites

In [ ]:
!pip install langchain --upgrade

In [ ]:
!pip install llama-index

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install sagemaker --upgrade

In [ ]:
!pip install boto3 --upgrade

In [ ]:
!pip install langchain-community --upgrade

## Restart Kernel

In [ ]:
#Restart Kernel after the installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)  

## Setup depedencies

In [1]:
#Check Python version is greater than 3.8 which is required by Langchain if you want to use Langchain
import sys
sys.version

'3.10.13 | packaged by conda-forge | (main, Oct 26 2023, 18:07:37) [GCC 12.3.0]'

In [2]:
assert sys.version_info >= (3, 8)

In [3]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

In [4]:
from llama_index import download_loader, SimpleDirectoryReader
from llama_index import VectorStoreIndex 
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index import LLMPredictor, ServiceContext
#import torch

import langchain
#from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [5]:
import os, json
from tqdm import tqdm
import pathlib
from pathlib import Path

In [6]:
import boto3
import sagemaker
session = boto3.Session()
sagemaker_session = sagemaker.Session()
studio_region = sagemaker_session.boto_region_name 
bedrock = session.client("bedrock-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [7]:
# Put your directory containing PDFs here
pdf_file_name = 'Amazon_Fire_TV_User_Guide.pdf'
pdf_path = f'pdfs/firetv/{pdf_file_name}'

In [8]:
index_directory= f'llama_indices'
index_path = f'{index_directory}/{pdf_file_name}'
pathlib.Path(index_directory).mkdir(parents=True, exist_ok=True)

## Load data with PDFReader

In [9]:
pdf_loader = download_loader('PDFReader')
loader = pdf_loader()
docs = loader.load_data(file=Path(pdf_path))

## Create Huggingface Embeddings and Embeddings model

In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings
emb = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
#emb = HuggingFaceEmbeddings()
emb_model = LangchainEmbedding(emb)


2024-01-26 20:52:20.008356: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
from langchain_community.llms.bedrock import Bedrock

#Creating Anthropic Claude
model_args= {'max_tokens_to_sample':200,'temperature':0}
llm = Bedrock(model_id="anthropic.claude-v1", client=bedrock, model_kwargs=model_args)
llm_predictor = LLMPredictor(llm=llm)

In [12]:
sc = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=emb_model)

LLMPredictor is deprecated, please use LLM instead.


## Persist with Vector store

In [13]:
index = VectorStoreIndex.from_documents(docs,service_context=sc)
index.storage_context.persist(persist_dir=index_path)

## Query LLM with Index

In [14]:
from llama_index import StorageContext, load_index_from_storage
stc = StorageContext.from_defaults(persist_dir=index_path)
index = load_index_from_storage(stc, service_context=sc)
engine = index.as_query_engine()

In [15]:
query = "How to setup Parental controls?"
resp = engine.query(query)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [16]:
#Print Query Response
print(resp.response)

 Here are the steps to set up Parental Controls on your Amazon Fire TV:

1. From the Home screen, select Settings, and then select Parental Controls.

2. Using your remote, press the Select button to turn parental controls On. 

3. Enter your Parental Controls PIN, and then select Next. Your PIN is the same PIN you use for other Amazon services such as Amazon Instant Video.

4. After you set a PIN, you can set one or more of the following settings:

•Require a PIN for all purchases: This will require a PIN for any purchases made on the Fire TV.

•Require a PIN for Amazon Instant Video only: This will require a PIN only for viewing content on Amazon Instant Video. 

•Block the ability to view or purchase certain content types, such as games, apps, or photos.

•Change your Parental Controls PIN:


In [17]:
#Browse through the source nodes
print(f'Total number of source nodes {len(resp.source_nodes)}')

Total number of source nodes 2


In [18]:
for i, n in enumerate(resp.source_nodes):
    print(f'Node {i +1}') 
    print('-' * 20)    
    print(f'text : {n.node.text} \n')
    print(f'metadata : {n.node.metadata}\n')


Node 1
--------------------
text : Set Up Parental Controls
With Parental Controls, you can block purchases and restrict access to Amazon movies, TV shows,
games, apps, photos, and more.
Note: Parental controls do not restrict content in third-party applications. Parental controls for third-party applications
are determined by the app provider.
When entering the PIN, you will need to use an Amazon remote or the Fire TV Remote App. You cannot
use a third-party remote.
1. From the Home  screen, select Settings , and then select Parental Controls .
2. Using your remote, press the Select  button to turn parental controls On or Off.
3. Enter your Parental Controls PIN, and then select Next .
Your PIN is the same PIN you use for other Amazon services such as Amazon Instant Video.
Note: If you forget your PIN, go to Amazon Instant Video SettingsAmazon Instant Video Settings  (Amazon.com full
site)Amazon Instant Video Settings (https://www.amazon.com/video/settings) to reset your Parental Cont